In [1]:
from pathlib import Path
import sys

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

In [2]:
from aper_package.interactive_tool import InteractiveTool

In [3]:
tool = InteractiveTool()

In [4]:
tool.show()